In [ ]:
import plotly.express as px
import pandas as pd
# import vectorbtpro as vbt
import numpy as np
from vectorbtpro import * 

In [3]:
data = vbt.BinanceData.load('price_data_1m.pkl')
btc_data = data.get(symbol="BTCUSDT")
eth_data = data.get(symbol="ETHUSDT")
eth_data.index = pd.to_datetime(eth_data.index)

filename = 'astra.xlsx'
transactions = pd.read_excel(filename, sheet_name='Sheet1', index_col='datetime', parse_dates=True)
transactions.index = pd.to_datetime(transactions.index, format='ISO8601')
# transactions.index = transactions.index.round("min")
transactions = transactions.sort_index()

In [88]:
start = '2024-01-31'
end = '2024-02-08'

eth_data = eth_data.loc[start:end]
transactions = transactions.loc[start: end]


In [89]:
deposits = transactions[transactions["direction"] == "allocation"].copy()

funding_fees = transactions[transactions["direction"] == "funding"].copy()
total_fees = funding_fees['fees'].sum()

trades = transactions[(transactions["direction"] !=  "allocation") & (transactions["direction"] != "funding")].copy()
trades['trade_direction'] = trades.apply(lambda x: -1 if (x['direction'] == 'open-short' or x['direction'] == 'close-long') else 1, axis=1)
trades['trade_quantity'] = trades['trade_direction'] * trades['quantity']


number_of_trades = len(trades)
avg_fee_added = total_fees / number_of_trades
trades['adj fees'] = trades['fees'] + avg_fee_added

print(f"Total Trades {number_of_trades}")
print(f"Total Funding Fee {total_fees}")
print(f"Fee added to each trade {avg_fee_added}")

print(trades)


Total Trades 6
Total Funding Fee 0.0
Fee added to each trade 0.0
                                  id acccount_id              strategy  \
datetime                                                                 
2024-02-01 14:18:42.223000+00:00   1       Astra     Yosemite ETH Fast   
2024-02-01 14:18:45.439000+00:00   2       Astra  Yosemite ETH Fast V3   
2024-02-01 14:35:54.289000+00:00   3       Astra     Yosemite ETH Fast   
2024-02-01 14:36:40.475000+00:00   4       Astra  Yosemite ETH Fast V3   
2024-02-01 17:20:26.948000+00:00   5       Astra     Yosemite ETH Slow   
2024-02-01 17:27:57.278000+00:00   6       Astra     Yosemite ETH Slow   

                                                           position_id  \
datetime                                                                 
2024-02-01 14:18:42.223000+00:00  f96491f9-b38c-4604-9aea-33c559ffcbcb   
2024-02-01 14:18:45.439000+00:00  0fe822f3-2c0b-4371-80e7-3ac24fd0a6ae   
2024-02-01 14:35:54.289000+00:00  f96491f9-b38

In [104]:
deposits.index = deposits.index.round("min")
eth_data['deposits'] = 0

for index, row in deposits.iterrows():
    eth_data.at[index, 'deposits'] += row['quantity']


depos = eth_data[eth_data['deposits'] != 0]
print(depos)

# print(eth_data)

                              Open     High     Low    Close    Volume  \
Open time                                                                
2024-01-31 23:43:00+00:00  2285.99  2285.99  2285.2  2285.21  126.6518   

                            Quote volume  Trade count  Taker base volume  \
Open time                                                                  
2024-01-31 23:43:00+00:00  289453.756728          268            65.2596   

                           Taker quote volume  deposits  
Open time                                                
2024-01-31 23:43:00+00:00       149145.776184    100000  


In [93]:

# Step 1: Identify duplicate timestamps
duplicates = trades.index.duplicated(keep='first')

# Step 2: Adjust duplicates by adding 1 millisecond to each subsequent duplicate
adjustment = pd.to_timedelta(1, unit='ms')  # 1 millisecond
for i in range(len(duplicates)):
    if duplicates[i]:  # If this is a duplicate
        # Add an adjustment based on how many previous duplicates there have been
        trades.index.values[i] += adjustment

# Verify if the operation was successful
print(trades.index.is_unique)  # This should now print 'True'

True


In [94]:
import datetime
import pytz

fake_data = eth_data.copy()

# Convert the datetime to match the DataFrame's index precision
deposit1 = datetime.datetime(year=2024, month=1, day=31, hour=0, minute=1, second=0)
deposit2 = datetime.datetime(year=2024, month=1, day=31, hour=0, minute=4, second=0)

# Now use the corrected datetime index to set the value
fake_data['deposits'] = 0
fake_data.at[deposit1.replace(tzinfo=pytz.utc), "deposits"] = 100
fake_data.at[deposit2.replace(tzinfo=pytz.utc), "deposits"] = 100

# Verify the change
fake_data

fake_trades = {
    'datetime' : [
        datetime.datetime(year=2024, month=1, day=31, hour=0, minute=2, second=0).replace(tzinfo=pytz.utc), 
        datetime.datetime(year=2024, month=1, day=31, hour=0, minute=3, second=0).replace(tzinfo=pytz.utc),
        datetime.datetime(year=2024, month=1, day=31, hour=0, minute=5, second=0).replace(tzinfo=pytz.utc),
        datetime.datetime(year=2024, month=1, day=31, hour=0, minute=6, second=0).replace(tzinfo=pytz.utc)
    ],
    'size': [1, -1, 1, -1],
    # 'price': [1000, 2000, 1000, 2000],
    'fees': [0, 0, 0, 0]
}

fake_trades_df = pd.DataFrame(fake_trades)
fake_trades_df['datetime'] = pd.to_datetime(fake_trades_df['datetime'])
fake_trades_df = fake_trades_df.set_index("datetime")

fake_data = fake_data.head(10)

fake_data['Close'] = [100, 100, 100, 101, 100, 100, 101, 102, 103, 102]
# print(fake_data[['Close', 'deposits']])
# fake_trades_df

In [102]:
eth_data['deposits'] = 0

In [106]:

pf = vbt.Portfolio.from_orders(
    close = eth_data.Close, # Note, here we are using the minutely data for ETH
    size = trades['trade_quantity'],  # Here we are only looking at the trades dataframe
    price = trades['price'],
    size_type = 'amount',
    fixed_fees = trades['fees'],
    init_cash = 1,
    leverage=4,
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,
    cash_deposits=eth_data['deposits'],
    cash_deposits_as_input = True,
    freq = '1T',
)
# pf = vbt.Portfolio.from_orders(
#     close = eth_trimmed.Close, # Note, here we are using the minutely data for ETH
#     size = fake_trades_df['size'],  # Here we are only looking at the trades dataframe
#     # price = fake_trades_df['price'],
#     size_type = 'amount',
#     fixed_fees = fake_trades_df['fees'],
#     init_cash = 0,
#     # leverage=4,
#     # leverage_mode=vbt.pf_enums.LeverageMode.Eager,
#     cash_deposits=eth_trimmed['deposits'],
#     cash_deposits_as_input = False,
#     freq = '1T',
# )

# print(pf.cash.to_markdown())
pf.equity_curve
pf.assets.to_csv("assets.csv")
pf.cash.to_csv("cash.csv")
eth_data['deposits'].to_csv("deposts.csv")
print(pf.stats())
pf.plot(title=f"yay").show()

Start                         2024-01-31 00:00:00+00:00
End                           2024-02-02 23:59:00+00:00
Period                                  3 days 00:06:00
Start Value                                         1.0
Min Value                                           1.0
Max Value                        13884521315345745920.0
End Value                        13884521315345747968.0
Cash Deposits                    13884521315345747968.0
Total Return [%]                                    0.0
Benchmark Return [%]                           -1.60627
Total Time Exposure [%]                        0.670365
Max Gross Exposure [%]                       144.179415
Max Drawdown [%]                               0.265876
Max Drawdown Duration                   1 days 09:21:00
Total Orders                                          6
Total Fees Paid                              176.321568
Total Trades                                          3
Win Rate [%]                                    

In [100]:
eth_data[eth_data['deposits'] != 0]
# print(eth_data['deposits'].to_markdown())
print(eth_data.tail())

                              Open     High      Low    Close    Volume  \
Open time                                                                 
2024-02-02 23:55:00+00:00  2308.18  2308.53  2308.18  2308.53   34.1138   
2024-02-02 23:56:00+00:00  2308.52  2308.53  2307.99  2307.99  207.4119   
2024-02-02 23:57:00+00:00  2308.00  2308.22  2307.99  2308.21   53.8197   
2024-02-02 23:58:00+00:00  2308.21  2309.00  2308.21  2309.00  121.7513   
2024-02-02 23:59:00+00:00  2308.99  2309.43  2308.99  2309.06   50.2101   

                            Quote volume  Trade count  Taker base volume  \
Open time                                                                  
2024-02-02 23:55:00+00:00   78745.841418          189            29.8837   
2024-02-02 23:56:00+00:00  478760.120990          278            47.1409   
2024-02-02 23:57:00+00:00  124216.998583          141            33.1835   
2024-02-02 23:58:00+00:00  281087.118042          270            74.3840   
2024-02-02 23:59:0